In [12]:
! py -m pip install mlxtend --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.4.1 requires packaging<22.0,>=0.21, but you have packaging 23.2 which is incompatible.



  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 1.4/1.4 MB 23.2 MB/s eta 0:00:00
   ---------------------------------------- 11.1/11.1 MB 18.8 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.1.7
    Uninstalling mlxtend-0.1.7:
      Successfully uninstalled mlxtend-0.1.7


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from sklearn.model_selection import train_test_split
from datetime import date, timedelta

fake = Faker()
Faker.seed(42)
np.random.seed(42)
random.seed(42)

num_customers = 10000

def generate_credit_score(income):
    base = np.interp(income, [20000, 200000], [500, 800])
    noise = np.random.normal(loc=0, scale=50)
    return int(np.clip(base + noise, 300, 850))

def generate_employment_status(age):
    if age < 18:
        return "Unemployed"
    elif age < 30:
        return random.choices(
            ['Full-Time', 'Part-Time', 'Self-Employed', 'Student'],
            weights=[0.4, 0.2, 0.1, 0.3]
        )[0]
    elif age >= 60:
        return random.choices(
            ['Retired', 'Self-Employed', 'Unemployed'],
            weights=[0.7, 0.2, 0.1]
        )[0]
    else:
        return random.choices(
            ['Full-Time', 'Part-Time', 'Self-Employed', 'Unemployed'],
            weights=[0.6, 0.15, 0.15, 0.1]
        )[0]

def generate_account_type():
    return random.choices(
        ['Savings', 'Checking', 'Both'],
        weights=[0.2, 0.3, 0.5]
    )[0]

def generate_customer_since(age):
    today = date.today()
    if age < 20:
        return today - timedelta(days=365)  # 1 year
    elif age < 30:
        return today - timedelta(days=random.randint(365, 5*365))  # up to 5 years
    else:
        return today - timedelta(days=random.randint(365, 10*365))  # up to 10 years

# Generate customers
customers = []
for i in range(1, num_customers + 1):
    is_male = random.choice([True, False])
    first_name = fake.first_name_male() if is_male else fake.first_name_female()
    gender = "M" if is_male else "F"
    last_name = fake.last_name()
    dob = fake.date_of_birth(minimum_age=18, maximum_age=75)
    age = int((pd.Timestamp.now().date() - pd.to_datetime(dob).date()).days / 365)

    income = np.random.randint(20, 200) * 1000  # in increments of 1000
    credit_score = generate_credit_score(income)
    marital_status = random.choice(['Married', 'Single', 'Divorced', 'Widowed']) if age >= 18 else 'Single'

    customers.append({
        "Customer_ID": f"CUST{i:05d}",
        "First_Name": first_name,
        "Last_Name": last_name,
        "Date_of_Birth": dob,
        "Age": age,
        "Gender": gender,
        "Marital_Status": marital_status,
        "Email": fake.email(),
        "Phone": fake.phone_number(),
        "Address": fake.address().replace("\n", ", "),
        "City": fake.city(),
        "State": fake.state(),
        "Postal_Code": fake.postcode(),
        "Country": "USA",
        "Employment_Status": generate_employment_status(age),
        "Annual_Income": income,
        "Credit_Score": credit_score,
        "Account_Type": generate_account_type(),
        "Customer_Since": generate_customer_since(age)
    })

df_customers = pd.DataFrame(customers)

# Split into train and test
# train_df, test_df = train_test_split(df_customers, test_size=0.2, random_state=42)

# # Preview
# print(train_df.head())

# # Uncomment to save
# train_df.to_csv("customers_train.csv", index=False)
# test_df.to_csv("customers_test.csv", index=False)

df_customers.to_csv("customers.csv", index=False)


     Customer_ID First_Name Last_Name Date_of_Birth  Age Gender  \
9254   CUST09255     Steven     Singh    1957-05-17   67      M   
1561   CUST01562      Eddie      Bush    1977-08-26   47      M   
1670   CUST01671      Diane     Perry    1966-11-11   58      F   
6087   CUST06088      Emily  Martinez    1978-11-08   46      F   
6669   CUST06670    William    Morrow    2006-05-14   18      M   

     Marital_Status                     Email                Phone  \
9254        Widowed  ellisanthony@example.net           2005344588   
1561        Married  mariahgibson@example.org   889.926.7013x61916   
1670        Widowed     gateskyle@example.net    516-531-6641x7000   
6087       Divorced       karnold@example.com    283.523.5976x2995   
6669        Widowed       obailey@example.com  +1-491-818-7979x573   

                                                Address             City  \
9254  641 Larsen Harbors Apt. 904, Thomasbury, DE 72836       Coxchester   
1561       77594 Kenneth

In [6]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()
np.random.seed(42)
random.seed(42)

# Load customer IDs from combined train + test
customer_ids = [f"CUST{i:05d}" for i in range(1, 10001)]

# Define life events
life_events = {
    "New Job": ["Salary - Google LLC", "Uber Eats", "BestBuy", "Starbucks"],
    "Marriage": ["Wedding Planner", "Zales Jewelry", "Airbnb Honeymoon", "Flower Co"],
    "Home Purchase": ["Home Depot", "Furniture World", "Mortgage Payment", "Lowe's"],
    "Retirement": ["Pension Fund Credit", "Leisure Travel", "Pharmacy", "Garden Supplies"],
    "Childbirth": ["BabyGap", "Target", "Pediatric Hospital", "Mothercare"],
    "College": ["University Bookstore", "Tuition Fee", "Laptop Purchase", "Dorm Essentials"]
}

life_event_categories = {
    "New Job": ["Salary", "Food", "Electronics", "Coffee"],
    "Marriage": ["Wedding", "Jewelry", "Travel", "Flowers"],
    "Home Purchase": ["Home Improvement", "Furniture", "Mortgage", "DIY"],
    "Retirement": ["Pension", "Travel", "Healthcare", "Gardening"],
    "Childbirth": ["Baby", "Retail", "Medical", "Baby Gear"],
    "College": ["Education", "Tuition", "Electronics", "Student Supplies"]
}

# Assign 3,000 customers to life events (500 per event)
life_event_customers = {}
all_life_event_ids = random.sample(customer_ids, 3000)
remaining_ids = set(customer_ids) - set(all_life_event_ids)

split_event_ids = np.array_split(all_life_event_ids, 6)
life_event_keys = list(life_events.keys())

for i, event in enumerate(life_event_keys):
    life_event_customers[event] = list(split_event_ids[i])

transactions = []
txn_id = 1

def generate_random_transaction(customer_id):
    merchant = random.choice([
        "Walmart", "Shell Gas", "Grocery Town", "Subway", "Pharmacy Hub", "Local Market"
    ])
    category = random.choice(["Grocery", "Fuel", "Dining", "Pharmacy", "Retail"])
    amount = round(random.uniform(10, 200), 2)
    description = f"Purchase at {merchant}"
    date = fake.date_time_between(start_date="-3y", end_date="now")
    
    return {
        "Transaction_ID": f"TXN{txn_id:07d}",
        "Customer_ID": customer_id,
        "Timestamp": date,
        "Merchant": merchant,
        "Category": category,
        "Amount": amount,
        "Description": description
    }

# Generate life event transactions
for event, ids in life_event_customers.items():
    for customer_id in ids:
        num_txns = random.randint(3, 10)
        for _ in range(num_txns):
            merchant = random.choice(life_events[event])
            category = random.choice(life_event_categories[event])
            amount = round(random.uniform(100, 3000), 2) if "Salary" not in merchant and "Pension" not in merchant else round(random.uniform(3000, 12000), 2)
            description = f"{category} - {merchant}"
            date = fake.date_time_between(start_date="-3y", end_date="now")
            transactions.append({
                "Transaction_ID": f"TXN{txn_id:07d}",
                "Customer_ID": customer_id,
                "Timestamp": date,
                "Merchant": merchant,
                "Category": category,
                "Amount": amount,
                "Description": description
            })
            txn_id += 1

# Generate normal customer transactions
for customer_id in remaining_ids:
    num_txns = random.randint(3, 10)
    for _ in range(num_txns):
        txn = generate_random_transaction(customer_id)
        txn["Transaction_ID"] = f"TXN{txn_id:07d}"
        transactions.append(txn)
        txn_id += 1

# Create DataFrame
df_transactions = pd.DataFrame(transactions)

# Sort by timestamp for realism
df_transactions = df_transactions.sort_values(by="Timestamp").reset_index(drop=True)

print(df_transactions.head())

# Optional save
df_transactions.to_csv("transactions.csv", index=False)


  Transaction_ID Customer_ID           Timestamp         Merchant  \
0     TXN0046402   CUST07774 2022-04-18 05:54:56           Subway   
1     TXN0039211   CUST01840 2022-04-18 06:50:56     Grocery Town   
2     TXN0049956   CUST00310 2022-04-18 07:16:22     Pharmacy Hub   
3     TXN0046728   CUST02783 2022-04-18 07:24:42          Walmart   
4     TXN0018524   CUST01522 2022-04-18 07:55:21  Dorm Essentials   

      Category   Amount                    Description  
0       Retail    93.38             Purchase at Subway  
1       Retail    66.65       Purchase at Grocery Town  
2      Grocery    47.87       Purchase at Pharmacy Hub  
3       Retail   179.28            Purchase at Walmart  
4  Electronics  1939.09  Electronics - Dorm Essentials  


In [4]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import csv
# For reproducibility
np.random.seed(42)

total_customers = 10000

# Ensure each customer has at least one transaction.
base_customer_ids = np.arange(1, total_customers + 1)

# Decide on extra transactions (e.g., 5000 additional transactions)
extra_transactions = 10000
extra_customer_ids = np.random.randint(1, total_customers + 1, extra_transactions)

# Combine to get a list where every customer appears at least once.
customer_ids = np.concatenate([base_customer_ids, extra_customer_ids])
num_transactions = len(customer_ids)

# TransactionID: Sequential IDs starting from 5001
transaction_ids = np.arange(5001, 5001 + num_transactions)

# Extensive list of products and their variations that large banks typically offer
products = {
    'Mortgage': ['Fixed Rate', 'Adjustable Rate', 'Interest Only', 'Reverse Mortgage'],
    'Credit Card': ['Rewards', 'Cashback', 'Travel', 'Business', 'Secured'],
    'Checking Account': ['Premium', 'Basic', 'Senior', 'Interest Bearing'],
    'Savings Account': ['High Yield', 'Regular', 'Money Market', "Children's Savings"],
    'Certificate of Deposit': ['1-Year', '2-Year', '5-Year', '10-Year'],
    'Personal Loan': ['Secured', 'Unsecured', 'Debt Consolidation', 'Home Improvement'],
    'Auto Loan': ['New Car', 'Used Car', 'Refinance', 'Lease'],
    'Home Equity Loan': ['Fixed Rate', 'Variable Rate', 'HELOC'],
    'Investment Account': ['Managed', 'Self-Directed', 'Robo Advisor', 'Brokerage'],
    'Business Loan': ['Term Loan', 'Line of Credit', 'SBA Loan', 'Equipment Financing'],
    'Retirement Account': ['IRA', '401K', 'Roth IRA', 'Simple IRA'],
    'Student Loan': ['Federal', 'Private', 'Refinance', 'Consolidation']
}

# List of product names for random selection
product_list = list(products.keys())
chosen_products = np.random.choice(product_list, num_transactions)
variations = [np.random.choice(products[prod]) for prod in chosen_products]

# Generate random dates between 2020-01-01 and 2025-03-31
start_date = datetime(2020, 1, 1)
end_date = datetime(2025, 3, 31)
days_range = (end_date - start_date).days
random_days = np.random.randint(0, days_range + 1, num_transactions)
dates = [start_date + timedelta(days=int(day)) for day in random_days]
# Format dates as MM/DD/YYYY (e.g., "08/04/2020")
formatted_dates = [d.strftime("%m/%d/%Y") for d in dates]

# Create the DataFrame
df_transactions = pd.DataFrame({
    'TransactionID': transaction_ids,
    'CustomerID': customer_ids,
    'Product': chosen_products,
    'Variation': variations,
    'Date': formatted_dates
})

# Ensure Date column is explicitly string
df_transactions['Date'] = df_transactions['Date'].astype(str)

# Save to CSV with all fields quoted so that formatting is preserved
df_transactions.to_csv("../data/Transactions.csv", index=False, quoting=csv.QUOTE_ALL)
print("Mock Transactions data generated and saved as 'Transactions.csv'.")


Mock Transactions data generated and saved as 'Transactions.csv'.


In [17]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import csv

# For reproducibility
np.random.seed(42)

total_customers = 10000

# Ensure each customer has at least one transaction.
base_customer_ids = np.arange(1, total_customers + 1)
extra_transactions = 10000  # Increase extra transactions if needed
extra_customer_ids = np.random.randint(1, total_customers + 1, extra_transactions)
customer_ids = np.concatenate([base_customer_ids, extra_customer_ids])
num_transactions = len(customer_ids)

# TransactionID: Sequential IDs starting from 5001
transaction_ids = np.arange(5001, 5001 + num_transactions)

# Original products dictionary with variations
products = {
    'Mortgage': ['Fixed Rate', 'Adjustable Rate', 'Hybrid Rate', 'Interest Only', 'Balloon Payment', 'Reverse Mortgage'],
    'Credit Card': ['Rewards', 'Cashback', 'Travel', 'Business', 'Student', 'Secured', 'Balance Transfer'],
    'Checking Account': ['Premium', 'Basic', 'Student', 'Senior', 'Interest Bearing'],
    'Savings Account': ['High Yield', 'Regular', 'Money Market', "Children's Savings"],
    'Certificate of Deposit': ['3-Month', '6-Month', '1-Year', '2-Year', '5-Year', '10-Year'],
    'Personal Loan': ['Secured', 'Unsecured', 'Debt Consolidation', 'Home Improvement'],
    'Auto Loan': ['New Car', 'Used Car', 'Refinance', 'Lease'],
    'Home Equity Loan': ['Fixed Rate', 'Variable Rate', 'HELOC'],
    'Investment Account': ['Managed', 'Self-Directed', 'Robo Advisor', 'Brokerage'],
    'Business Loan': ['Term Loan', 'Line of Credit', 'SBA Loan', 'Equipment Financing'],
    'Retirement Account': ['IRA', '401K', 'Roth IRA', 'SEP IRA', 'Simple IRA'],
    'Student Loan': ['Federal', 'Private', 'Refinance', 'Consolidation']
}

# Define three product groups (fewer combinations per customer)
group1 = ['Mortgage', 'Credit Card', 'Business Loan']
group2 = ['Checking Account', 'Savings Account', 'Certificate of Deposit', 'Personal Loan']
group3 = ['Auto Loan', 'Home Equity Loan', 'Investment Account', 'Retirement Account', 'Student Loan']

groups = {1: group1, 2: group2, 3: group3}
all_products = group1 + group2 + group3  # complete list of products

# Assign each customer to one group at random
customer_group = {}
for cid in range(1, total_customers + 1):
    customer_group[cid] = np.random.choice([1, 2, 3])

# Generate chosen_products for each transaction:
# With 80% probability, choose from the customer's group.
chosen_products = []
expanded_customer_ids = []

for cid in customer_ids:
    if np.random.rand() < 0.9:
        n_products = np.random.choice([2, 3], p=[0.7, 0.3])
        prods = np.random.choice(groups[customer_group[cid]], size=n_products, replace=False)
        chosen_products.extend(prods)
        expanded_customer_ids.extend([cid] * n_products)
    else:
        prod = np.random.choice(all_products)
        chosen_products.append(prod)
        expanded_customer_ids.append(cid)

# Now they match
assert len(expanded_customer_ids) == len(chosen_products)

# TransactionID
transaction_ids = np.arange(5001, 5001 + len(expanded_customer_ids))

# Variations
chosen_variations = [np.random.choice(products[prod]) for prod in chosen_products]

# Dates
random_days = np.random.randint(0, days_range + 1, len(expanded_customer_ids))
dates = [start_date + timedelta(days=int(day)) for day in random_days]
formatted_dates = [d.strftime("%m/%d/%Y") for d in dates]

# Final DataFrame
df_transactions = pd.DataFrame({
    'TransactionID': transaction_ids,
    'CustomerID': expanded_customer_ids,
    'Product': chosen_products,
    'Variation': chosen_variations,
    'Date': formatted_dates
})

# Save
df_transactions['Date'] = df_transactions['Date'].astype(str)
df_transactions.to_csv("../data/Transactions.csv", index=False, quoting=csv.QUOTE_ALL)
print("Modified mock Transactions data generated and saved as 'Transactions.csv'.")

Modified mock Transactions data generated and saved as 'Transactions.csv'.


In [124]:
import pandas as pd
import numpy as np

# Load the Transactions data
df_transactions = pd.read_csv("../data/Transactions.csv")

# Generate a rating between 1.0 to 5.0 with slight skew toward higher values
# We'll use a clipped normal distribution
ratings = np.random.normal(loc=4.0, scale=0.7, size=len(df_transactions))
ratings = np.clip(ratings, 1.0, 5.0)
ratings = np.round(ratings, 1)

# Create the new DataFrame
df_ratings = df_transactions.copy()
df_ratings = df_ratings.drop(columns=["Date"])  # Remove Date
df_ratings["Rating"] = ratings  # Add Rating column

# Save to CSV
df_ratings.to_csv("../data/ProductVariationRatings.csv", index=False, quoting=1)  # quoting=1 = csv.QUOTE_ALL
print("ProductVariationRatings.csv generated successfully.")


ProductVariationRatings.csv generated successfully.


In [125]:
import pandas as pd

# Load the Transactions data
df_transactions = pd.read_csv("../data/Transactions.csv")

# Take a quick look at the data
print("Transactions Data Sample:")
print(df_transactions.head())


Transactions Data Sample:
   TransactionID  CustomerID             Product         Variation        Date
0           5001           1         Credit Card  Balance Transfer  04/30/2020
1           5002           1            Mortgage  Reverse Mortgage  11/15/2021
2           5003           2  Retirement Account           SEP IRA  04/12/2023
3           5004           2  Investment Account     Self-Directed  09/09/2023
4           5005           3         Credit Card           Rewards  03/28/2025


## Apriori rules

In [126]:
# Group transactions by CustomerID and aggregate unique products into a list (basket)
basket = df_transactions.groupby('CustomerID')['Product'] \
                          .apply(lambda x: list(set(x))) \
                          .reset_index()

print("\nBasket Sample:")
print(basket.head)



Basket Sample:
<bound method NDFrame.head of       CustomerID                                            Product
0              1                            [Mortgage, Credit Card]
1              2  [Investment Account, Auto Loan, Home Equity Lo...
2              3             [Business Loan, Mortgage, Credit Card]
3              4             [Business Loan, Mortgage, Credit Card]
4              5             [Business Loan, Mortgage, Credit Card]
...          ...                                                ...
9995        9996             [Investment Account, Home Equity Loan]
9996        9997  [Mortgage, Savings Account, Home Equity Loan, ...
9997        9998                          [Business Loan, Mortgage]
9998        9999  [Business Loan, Mortgage, Credit Card, Home Eq...
9999       10000             [Business Loan, Mortgage, Credit Card]

[10000 rows x 2 columns]>


In [127]:
from mlxtend.preprocessing import TransactionEncoder

# Transform the basket list into one-hot encoded format
te = TransactionEncoder()
te_array = te.fit(basket['Product']).transform(basket['Product'])
df_basket = pd.DataFrame(te_array, columns=te.columns_, index=basket['CustomerID'])
df_basket.reset_index(inplace=True)  # Optionally keep CustomerID as a column

print("\nOne-Hot Encoded Basket Sample:")
print(df_basket.head())



One-Hot Encoded Basket Sample:
   CustomerID  Auto Loan  Business Loan  Certificate of Deposit  \
0           1      False          False                   False   
1           2       True          False                   False   
2           3      False           True                   False   
3           4      False           True                   False   
4           5      False           True                   False   

   Checking Account  Credit Card  Home Equity Loan  Investment Account  \
0             False         True             False               False   
1             False        False              True                True   
2             False         True             False               False   
3             False         True             False               False   
4             False         True             False               False   

   Mortgage  Personal Loan  Retirement Account  Savings Account  Student Loan  
0      True          False              

In [128]:
from mlxtend.frequent_patterns import apriori, association_rules

# Drop CustomerID from the one-hot encoded DataFrame since it's not needed for Apriori
df_onehot = df_basket.drop(columns=['CustomerID'])

# Run the Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(df_onehot, min_support=0.005, use_colnames=True)

print("\nFrequent Itemsets Sample:")
print(frequent_itemsets)



Frequent Itemsets Sample:
    support                                           itemsets
0    0.2214                                        (Auto Loan)
1    0.2901                                    (Business Loan)
2    0.2590                           (Certificate of Deposit)
3    0.2481                                 (Checking Account)
4    0.2869                                      (Credit Card)
..      ...                                                ...
92   0.0591  (Auto Loan, Student Loan, Retirement Account, ...
93   0.0578  (Investment Account, Auto Loan, Student Loan, ...
94   0.0964  (Certificate of Deposit, Checking Account, Per...
95   0.0574  (Investment Account, Home Equity Loan, Student...
96   0.0406  (Investment Account, Student Loan, Retirement ...

[97 rows x 2 columns]


In [ ]:
# Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# print("\nAssociation Rules Sample:")
# print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

In [204]:
# Convert sets to lists so they can be JSON-serialized
rules_clean = rules.copy()
rules_clean['antecedents'] = rules_clean['antecedents'].apply(list)
rules_clean['consequents'] = rules_clean['consequents'].apply(list)

# Save as JSON Lines
rules_clean.to_json("../data/rules.json", orient="records", lines=True)


## Find Similar Customers

In [205]:
import pandas as pd

# Load JSON lines
rules = pd.read_json("../data/rules.json", lines=True)

# Convert antecedents/consequents back to sets
rules['antecedents'] = rules['antecedents'].apply(set)
rules['consequents'] = rules['consequents'].apply(set)


In [166]:
def recommend_next_products(customer_products, rules_df, min_conf, min_lift):
    recommendations = set()
    for _, row in rules_df.iterrows():
        antecedent = list(row['antecedents'])
        consequent = list(row['consequents'])
        if set(antecedent).issubset(set(customer_products)) and row['confidence'] >= min_conf and row['lift'] >= min_lift:
            recommendations.update(consequent)
    return list(recommendations)


In [131]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

def compute_customer_similarity(customers_df):
    # Select numeric and categorical features
    numeric_cols = ['Age', 'Income', 'Credit Score', 'Tenure']
    categorical_col = ['Segment']
    
    # Normalize numeric columns
    scaler = StandardScaler()
    numeric_scaled = scaler.fit_transform(customers_df[numeric_cols])
    
    # One-hot encode Segment
    encoder = OneHotEncoder(sparse_output=False)
    segment_encoded = encoder.fit_transform(customers_df[categorical_col])
    
    # Combine features into single vector
    combined_features = np.hstack((numeric_scaled, segment_encoded))
    
    # Compute cosine similarity matrix
    similarity_matrix = cosine_similarity(combined_features)
    
    # Return as DataFrame with CustomerID as index/columns
    sim_df = pd.DataFrame(similarity_matrix, 
                          index=customers_df['CustomerID'], 
                          columns=customers_df['CustomerID'])
    return sim_df


In [132]:
# Example: Customer 200 has Mortgage. We recommend Credit Card. Now find best Credit Card variation

# Load the Customers.csv first
customers_df = pd.read_csv("../data/Customers.csv")

# Compute cosine similarity between all customers using their attributes
sim_df = compute_customer_similarity(customers_df)


## Test set

In [208]:
import pandas as pd

# Load the real data from the data folder
df_customers = pd.read_csv("../data/Customers.csv")
df_transactions = pd.read_csv("../data/Transactions.csv")

# Get each customer's first product (sorted by transaction ID or date)
df_first_product = df_transactions.sort_values(by="TransactionID").groupby("CustomerID").first().reset_index()
df_first_product = df_first_product[['CustomerID', 'Product']]
df_first_product.rename(columns={'Product': 'MainProduct'}, inplace=True)

# Merge with customers
df_customer_test = pd.merge(df_customers, df_first_product, on="CustomerID", how="inner")

# Take any 100 customers for the test set
df_customer_test = df_customer_test.sample(n=100, random_state=42)

# Save to CSV
df_customer_test.to_csv("../data/customer_test.csv", index=False)

print("✅ Test set with 100 customers and their main product saved as 'customer_test.csv'")


✅ Test set with 100 customers and their main product saved as 'customer_test.csv'


## Recommend Product variation based on Similar Customers

In [133]:
def recommend_variation_with_customer_similarity(customer_id, product, df_ratings, sim_df, top_n_similar_users=10):
    if customer_id not in sim_df.index:
        return None

    # Get most similar customers
    similar_customers = sim_df.loc[customer_id].sort_values(ascending=False)
    similar_customers = similar_customers.drop(index=customer_id).head(top_n_similar_users).index.tolist()

    # Filter ratings by these similar customers for the product
    similar_ratings = df_ratings[(df_ratings['CustomerID'].isin(similar_customers)) &
                                 (df_ratings['Product'] == product)]

    if similar_ratings.empty:
        return None

    # Get highest-rated variation
    top_variation = (similar_ratings.groupby('Variation')['Rating']
                     .mean()
                     .sort_values(ascending=False)
                     .idxmax())
    return top_variation


In [134]:
recommend_variation_with_customer_similarity(
    customer_id=1, 
    product="Mortgage", 
    df_ratings=df_ratings, 
    sim_df=sim_df,top_n_similar_users=10
)

'Reverse Mortgage'

In [168]:
def full_recommendation(customer_id, df_transactions, rules_df, df_ratings, sim_df, min_conf, min_lift):
    # Step 1: Find what the customer already has
    customer_products = df_transactions[df_transactions['CustomerID'] == customer_id]['Product'].unique().tolist()
    
    # Step 2: Get recommended products using Apriori rules
    recommended_products = recommend_next_products(customer_products, rules_df, min_conf, min_lift)

    # Step 3: Recommend best variation using customer-based similarity
    recommendations = []
    for prod in recommended_products:
        best_variation = recommend_variation_with_customer_similarity(
            customer_id, prod, df_ratings, sim_df, 10
        )
        recommendations.append({
            'Product': prod,
            'Variation': best_variation or "Default"
        })

    return recommendations


In [207]:
df_transactions = pd.read_csv("../data/Transactions.csv")
# rules = pd.read_csv("../data/rules.csv")  # assuming you've saved your Apriori rules

recos = full_recommendation(
    customer_id=6,
    df_transactions=df_transactions,
    rules_df=rules,
    df_ratings=df_ratings,
    sim_df=sim_df, min_conf=0.6, min_lift=2
)

pd.DataFrame(recos)

,Product,Variation
0,Investment Account,Robo Advisor
1,Student Loan,Consolidation
2,Retirement Account,Simple IRA
3,Home Equity Loan,Fixed Rate
4,Auto Loan,Refinance
